In [ ]:
# two local csv files loaded from desktop and pandas-dataframed by using one click button
# merge two csv files
# write the merged file to csv format
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_1fd445c3b67620d21c5874b9c29e9fbf561cebb6','domain': {'id': '20b49a5877434f9486aa2a1d2fcdd21c'},
            'password': 'i&g.LVqvB1Hv_1~X'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140('newciidemotest', 'cii_input1.csv'))
df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140('newciidemotest', 'cii_input2.csv'))

# if the matching key, in this case, the INSURANCE_POLICY_ID was read in as a xbf format, you may need to reformat it as follows.
df_data_1.rename(columns={'﻿INSURANCE_POLICY_ID':'INSURANCE_POLICY_ID'}, inplace=True)
df_data_2.rename(columns={'﻿INSURANCE_POLICY_ID':'INSURANCE_POLICY_ID'}, inplace=True)

#df_data_2.head()
m = pd.merge(df_data_1, df_data_2, how = 'inner', on = 'INSURANCE_POLICY_ID')
#m.dtypes

m.to_csv('merge_ab_chyl4.csv', index = False)

In [ ]:
### even on the notebook with swift obj storage attached, if the user obtains access key, secret key, api endpoint and bucket name,
### the user can directly write the file out to the COS (e.g. CII COS bucket)

import os
import boto3

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

os.environ['AWS_ACCESS_KEY_ID'] = '68n7GYTRYfSDfeQRPlsW'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'NAymzuNHGudPpmmALLUP83ZtF6jrCth9AtEyrHly'
endpoint = 'https://s3-api.dal-us-geo.objectstorage.softlayer.net/'
bucket = 'tstf381898e6bfa44489d38ac781ff42ba8'
cos_51fd21f00433421d8060e69f51aec41c = boto3.resource('s3', endpoint_url=endpoint)

# write out the merged csv file to the target bucket in COS

data = open('merge_ab_chyl4.csv', 'rb')
cos_51fd21f00433421d8060e69f51aec41c.Bucket(bucket).put_object(Key='merge_ab_chyl4.csv', Body=data) 

In [ ]:
# checking my file objects in S3 object storage

import boto3

endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
cos_51fd21f00433421d8060e69f51aec41c = boto3.resource('s3', endpoint_url=endpoint)

for bucket in cos_51fd21f00433421d8060e69f51aec41c.buckets.all():
    print(bucket.name)
    for obj in bucket.objects.all():
        print("  - %s") % obj.key